In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/TsaiYunLi/RAG_miniproj_TsaiYunLi/main/HIMYM_chacters.csv')
data = df.to_dict('records')
len(data)

20

In [2]:
! pip install tabula-py

In [3]:
df.head()

,name,sex,age,relation,personality
0,Ted Mosby,Male,35,"Main character, architect, and college friend ...",Ted Mosby is an architect with a romantic and ...
1,Robin Scherbatsky,Female,32,News anchor and Ted's ex-girlfriend,Robin Scherbatsky is a fiercely independent an...
2,Marshall Eriksen,Male,35,"Lawyer, married to Lily Aldrin, college friend...",Marshall Eriksen is a kind-hearted and optimis...
3,Lily Aldrin,Female,34,"Kindergarten teacher, married to Marshall Erik...",Lily Aldrin is a vibrant and creative kinderga...
4,Barney Stinson,Male,36,"Ted's friend, known for his womanizing and leg...",Barney Stinson is a charismatic and wealthy ma...


In [4]:
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 29.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.


In [5]:
!pip install sentence_transformers

In [6]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [9]:
# Create collection to store the character records
qdrant.recreate_collection(
    collection_name="HIMYM_characters",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<ipython-input-9-abd7ff00d47a>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [10]:
# vectorize!
qdrant.upload_points(
    collection_name="HIMYM_characters",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["personality"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [11]:
user_prompt = "I am a 28-year-old Asian women new to New York who works as a junior AI engineer. I am an introvert. I love arts, animals, sports, and nature. Who are my potential good friends?"

In [12]:
# Search time for HIMYM_characters!

hits = qdrant.search(
    collection_name="HIMYM_characters",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Ted Mosby', 'sex': 'Male', 'age': 35, 'relation': 'Main character, architect, and college friend of Marshall and Lily', 'personality': "Ted Mosby is an architect with a romantic and idealistic personality. He believes in finding the one true love and often shares his elaborate and intricate romantic stories with his friends. Despite numerous heartbreaks, Ted remains optimistic about love and is willing to go to great lengths to find his soulmate. He is thoughtful and kind, always ready to support his friends. Ted's passion for architecture often leads him to appreciate the finer details in life. He can be overly analytical at times, but his loyalty and dedication to his friends are unmatched."} score: 0.21728344326011415
{'name': 'Victoria', 'sex': 'Female', 'age': 30, 'relation': "Ted's girlfriend, baker", 'personality': 'Victoria is a talented baker with a kind and gentle personality. She has a sweet and caring nature, which makes her a perfect match for Ted. Victoria is am

In [13]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [14]:
!pip install openai

In [15]:
'''
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://118.167.11.136:85/", # "http://<Your api-server IP>:port"
    #local IP is http://127.0.0.1:8080/ #still didn't work using NAT
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are a chatbot, a super fan of the TV series 'How I Met Your Mothers'. Your top priority is to use your knowledge about its characters to answer anoher fan's questions."},
        {"role": "user", "content": "Suggest me a young, male character who welcomes foreigners."},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)
'''

'\n# Now time to connect to the local large language model\nfrom openai import OpenAI\nclient = OpenAI(\n    base_url="http://118.167.11.136:85/", # "http://<Your api-server IP>:port"\n    #local IP is http://127.0.0.1:8080/ #still didn\'t work using NAT\n    api_key = "sk-no-key-required"\n)\ncompletion = client.chat.completions.create(\n    model="LLaMA_CPP",\n    messages=[\n        {"role": "system", "content": "You are a chatbot, a super fan of the TV series \'How I Met Your Mothers\'. Your top priority is to use your knowledge about its characters to answer anoher fan\'s questions."},\n        {"role": "user", "content": "Suggest me a young, male character who welcomes foreigners."},\n        {"role": "assistant", "content": str(search_results)}\n    ]\n)\nprint(completion.choices[0].message)\n'

In [16]:
'''
from transformers import AutoModelForCausalLM, AutoTokenizer # local

# Load a HuggingFace model
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define the conversation
system_message = "You are a chatbot, a super fan of the TV series 'How I Met Your Mother'. Your top priority is to use your knowledge about its characters to answer another fan's questions."
user_message = "Suggest me a young, male character who welcomes foreigners."

# Prepare the input
input_text = f"<s>[SYSTEM]{system_message}\n[USER]{user_message}\n[ASSISTANT]"

inputs = tokenizer(input_text, return_tensors="pt")

# Generate the response
outputs = model.generate(inputs["input_ids"], max_new_tokens=50)

# Decode the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)
'''

'\nfrom transformers import AutoModelForCausalLM, AutoTokenizer # local\n\n# Load a HuggingFace model\nmodel_name = "meta-llama/Llama-2-7b-chat-hf"\ntokenizer = AutoTokenizer.from_pretrained(model_name)\nmodel = AutoModelForCausalLM.from_pretrained(model_name)\n\n# Define the conversation\nsystem_message = "You are a chatbot, a super fan of the TV series \'How I Met Your Mother\'. Your top priority is to use your knowledge about its characters to answer another fan\'s questions."\nuser_message = "Suggest me a young, male character who welcomes foreigners."\n\n# Prepare the input\ninput_text = f"<s>[SYSTEM]{system_message}\n[USER]{user_message}\n[ASSISTANT]"\n\ninputs = tokenizer(input_text, return_tensors="pt")\n\n# Generate the response\noutputs = model.generate(inputs["input_ids"], max_new_tokens=50)\n\n# Decode the response\nresponse = tokenizer.decode(outputs[0], skip_special_tokens=True)\nprint(response)\n'

In [24]:
from huggingface_hub import InferenceClient # API

# Hugging Face API Key
#api_key = "..." # hidden my api key

# Initialize Hugging Face Inference Client for GPT-2
client = InferenceClient(model="tiiuae/falcon-7b-instruct", token=api_key)

# Simulate a user query
user_prompt = "I am a 28-year-old Asian women new to New York who works as a junior AI engineer. I am an introvert. I love arts, animals, sports, and nature. Which 'How I Met Your Mother' characters are my potential good friends?"

# Step 1: Retrieve Relevant Knowledge from Qdrant
hits = qdrant.search(
    collection_name="HIMYM_characters",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

# Combine retrieved knowledge into a single context
search_results = [hit.payload for hit in hits]
retrieved_knowledge = " ".join([str(result) for result in search_results])
print(f"Retrieved Knowledge: {retrieved_knowledge}")

# Step 2: Define the Input for the LLM
# Combine the system message, retrieved knowledge, and user prompt
system_message = (
    "You are a chatbot, a super fan of the TV series 'How I Met Your Mother'. "
    "Your top priority is to use your knowledge about its characters to answer another fan's questions. "
    "Here is some retrieved knowledge: "
)
input_text = f"{system_message}{retrieved_knowledge}\nUser: {user_prompt}\nAssistant:"

# Step 3: Generate a Response using the LLM
try:
    response = client.text_generation(input_text, max_new_tokens=50)
    print("Assistant:", response)
except Exception as e:
    print(f"An error occurred: {e}")


Retrieved Knowledge: {'name': 'Victoria', 'sex': 'Female', 'age': 30, 'relation': "Ted's girlfriend, baker", 'personality': 'Victoria is a talented baker with a kind and gentle personality. She has a sweet and caring nature, which makes her a perfect match for Ted. Victoria is ambitious and passionate about her career, often balancing her professional aspirations with her personal life. Despite the ups and downs in her relationship with Ted, Victoria remains a positive and resilient character.'} {'name': 'Tracy McConnell', 'sex': 'Female', 'age': 28, 'relation': "Ted's wife, mother of his children", 'personality': "Tracy McConnell is Ted's wife and the mother of his children. She is a kind-hearted and loving individual who shares Ted's romantic ideals. Tracy's musical talent and her ability to connect with others make her a perfect match for Ted. Her presence brings joy and warmth to Ted's life, and her character adds depth to the series' narrative."} {'name': 'Nora', 'sex': 'Female', 